In [1]:
# %% [Imports]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_groq import ChatGroq
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage
from bs4 import BeautifulSoup
import requests
import os
from dotenv import load_dotenv

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
# %% [Load API Keys]
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

# Initialize LLM
llm = ChatGroq(groq_api_key=groq_api_key, model_name="Llama3-8b-8192")

In [ ]:
# %% [Step 1: Crawl All Links in the Website]
def extract_links(url):
    """Extracts all hyperlinks from a given webpage."""
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    # print(soup)

    # Find all hyperlinks on the page
    links = set()
    for link in soup.find_all("a", href=True):
        href = link["href"]
        # Convert relative links to absolute
        if href.startswith("/"):
            href = url.rstrip("/") + href
        if href.startswith("http"):  # Only store valid URLs
            links.add(href)
    
    return list(links)

# Get all links on the main page
base_url = "https://docs.smith.langchain.com/"
all_links = extract_links(base_url)

print(f"Found {len(all_links)} links to process.")

<!DOCTYPE html>

<html class="docs-wrapper plugin-docs plugin-id-default docs-version-current docs-doc-page docs-doc-id-index" data-has-hydrated="false" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="Docusaurus v3.5.2" name="generator"/>
<title data-rh="true">Get started with LangSmith | 🦜️🛠️ LangSmith</title><meta content="width=device-width,initial-scale=1" data-rh="true" name="viewport"/><meta content="summary_large_image" data-rh="true" name="twitter:card"/><meta content="https://docs.smith.langchain.com/img/langsmith-preview.png" data-rh="true" property="og:image"/><meta content="https://docs.smith.langchain.com/img/langsmith-preview.png" data-rh="true" name="twitter:image"/><meta content="https://docs.smith.langchain.com/" data-rh="true" property="og:url"/><meta content="en" data-rh="true" property="og:locale"/><meta content="en" data-rh="true" name="docusaurus_locale"/><meta content="en" data-rh="true" name="docsearch:language"/><meta content="current" data-r

In [15]:
for i in range(len(all_links)):
    if "https://docs.smith.langchain.com/observability" in all_links[i]:
        print(all_links[i], type(all_links[i]))

https://docs.smith.langchain.com/observability/how_to_guides/trace_with_langchain <class 'str'>
https://docs.smith.langchain.com/observability <class 'str'>
https://docs.smith.langchain.com/observability/how_to_guides/tracing/trace_with_langgraph <class 'str'>
https://docs.smith.langchain.com/observability/how_to_guides/dashboards <class 'str'>
